*This jupyter notebook is part of Arizona State University's course CAS 523 (Methods for Complex Systems Science: Statistics and Dimensionality Reduction) and was written by Bryan Daniels.  It was last updated September 28, 2022.*

*This assignment uses data gathered by Ying Wang and Robert E. Page, Jr. at Arizona State University.*

# Performing model selection using the Bayesian information criterion

In this assignment, we will look again at the honey bee gene expression data that we previously analyzed using PCA.  Now going beyond a descriptive analysis, we will attempt to find a generative model that can output data that matches the statistics we measure in the data.  

In this case, the inferred model will take the form of a probability distribution of gene expression values, predicting which combinations of gene expression are more or less likely.  A typical challenge when inferring such models is selecting the best form of model—in this case, selecting the form of the model sets the allowed shapes of the distribution.  Will a simple model suffice, or do we need to include more detail?  Here, we will use the "Bayesian information criterion" as a measure to decide which model is best.

## Get set up and load the data

**NOTE:** As in the previous PCA assignment, the dataset that we use in this assignment is not yet publicly available, but I have permission to share it with students.  For this reason, I do not include the data file in the public github repository.  Instead, it is available for download on Canvas.  Please follow the link on the Canvas assignment page to download the **two** files `Wang_Page_nanostring_data_2016_Day_1.csv` and `Wang_Page_nanostring_data_2016_Day_15.csv` and place them in the folder `data/WangPage2016/` in your own copy of the github repository.  Please do not share these data outside of this class.

Let's first load some standard packages:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 18}) # increases font size on plots
from pathlib import Path # to handle file paths across all operating systems

And some more specialized functions that we'll make use of:

In [ ]:
from scipy import stats
from sklearn.decomposition import PCA
from helpers.prettyPlotting import scatter1D # custom 1D scatter plot
from helpers.landau import LandauDistributionPDF # form of bimodal distribution that we fit

And now we'll load the gene expression data.  In the previous PCA assignment, we only looked at gene expression of bees at age 15 days.  Here, we'll also separately load gene expression profiles for bees at age 1 day.

In [ ]:
dataPathDay1 = Path('data/WangPage2016/Wang_Page_nanostring_data_2016_Day_1.csv')
expressionDataDay1 = pd.read_csv(dataPathDay1).drop(columns=['Age'])

dataPathDay15 = Path('data/WangPage2016/Wang_Page_nanostring_data_2016_Day_15.csv')
expressionDataDay15 = pd.read_csv(dataPathDay15).drop(columns=['Age'])

To remind ourselves what these data look like:

In [ ]:
expressionDataDay1

For each age, we have measurements of the expression levels of 91 genes for each of 16 bees.

## Reduce dimensionality

Recall that, in the PCA assignment, we found what looked like two different groups of bees when we looked along the first principal component of the gene expression data.  Let's first reduce the dimensionality of our data by focusing only on this first component.  We'll do the calculation separately for bees of age 1 day and 15 days:

In [ ]:
pcaProjectionsDay1 = PCA(n_components=1).fit_transform(expressionDataDay1)
pcaProjectionsDay15 = PCA(n_components=1).fit_transform(expressionDataDay15)

Recall also *why* it makes sense to look only along the first principal component: If we think the bees' gene expression is separating out into two distinct clusters, then the genes involved in that separation will have larger variance.  Focusing along the dimension with maximal variance—the first principal component—will highlight any such differences.

The following makes a one-dimensional scatter plot of the Day 15 data as in our previous assignment:

In [ ]:
scatter1D(pcaProjectionsDay15)
plt.xlabel('Distance along first principal component')
plt.title('Day 15');

❓ **Make a 1D scatter plot for bees of age 1 day.  How do the two ages of bees look different?  By eye, would you guess that one or the other, or both, ages display distinct clusters of gene expression?**

In [ ]:
# ✳️ **Answer:**

In the rest of the exercise, we will use statistical model selection to more precisely check our intuitive guess.

## Find best-fit Gaussian (unimodal)

The simplest guess for the distribution of gene expression along the principal component would be a Gaussian—the classic bell-shaped curve corresponding to a single continuous cluster.  This is called a unimodal distribution because it only has one peak.

Recall from our exercise on quantifying uncertainty how to fit a Gaussian distribution to data:

In [ ]:
paramsNormalDay15 = stats.norm.fit(pcaProjectionsDay15)

We can use the fit parameters to plot the probability density alongside a histogram of the data:

In [ ]:
# plot the best-fit Gaussian
plt.hist(pcaProjectionsDay15,bins=10,density=True)
xs = np.linspace(-15,15,100)
plt.plot(xs,[stats.norm.pdf(x,*paramsNormalDay15) for x in xs],lw=5)
plt.xlabel('Distance along first component')
plt.ylabel('Probability density')
plt.title('Day 15, Gaussian fit');

❓ **Looking at this plot, how well would you say the Gaussian does at fitting the data?**

✳️ **Answer:** 

Recall that the function `nnlf` returns the negative log-likelihood of the data given the model, so `-nnlf` is the standard log-likelihood:

In [ ]:
gaussianLogLDay15 = -stats.norm.nnlf(paramsNormalDay15,pcaProjectionsDay15)[0]
print("The log-likelihood for the Gaussian model on Day 15 data is {}.".format(gaussianLogLDay15))

We will use this next in comparing to a more complicated distribution with two bumps.

## Find best-fit "Landau" distribution (bimodal)

In a particular mathematical sense that I won't describe in detail here, the simplest distribution with *two* peaks (bimodal) is what I will call the "Landau" distribution.  This distribution corresponds to a very simplistic model of phase transitions in statistical physics. Here's the form of the distribution in case you are interested:

$$
p(x) = \frac{1}{Z} \exp \left( -\frac{c}{2} (x-\mu)^2 - \frac{d}{4} (x-\mu)^4 \right),
$$

where $Z$ is a normalization constant.  

The Landau distribution has three parameters ($\mu$, $c$, and $d$) compared to the Gaussian model's two parameters ($\mu$ and $\sigma$).  Note that the Gaussian is a special case of the Landau distribution: setting $d=0$ gives the usual form of a Gaussian.  So this model can have a single peak or two peaks depending on the values of the parameters.

The Landau distribution is not standard enough to be implemented in the packages we use here, so I've done the fitting of parameters for you.  We load these parameters here:

In [ ]:
landauFitParameters = pd.read_csv(Path('data/WangPage2016/landau_fit_parameters.csv'),index_col='age')

We can plot the fit distribution using the function `LandauDistributionPDF` that I wrote:

In [ ]:
# plot the best-fit Landau distribution
age = 15 # age of bees in days
params = landauFitParameters.loc[age]
plt.hist(pcaProjectionsDay15,bins=10,density=True)
xs = np.linspace(-15,15,100)
plt.plot(xs,LandauDistributionPDF(xs,params['mu'],params['c'],params['d']),lw=5)
plt.xlabel('Distance along first component')
plt.ylabel('Probability density')
plt.title('Day 15, Landau fit');

This looks like a much better fit!  Still, it's not exactly right—but then again, we only have 16 datapoints, so we don't expect the fit to be exactly right.  How do we decide if it's better enough compared to the unimodal distribution?  This is a job for statistical model selection!

## Compute and interpret the Bayesian information criterion

First, let's look at the log-likelihood of the data given the Landau model.  I also have this pre-saved:

In [ ]:
landauLogLDay15 = landauFitParameters.loc[15]['log-likelihood']
print("The log-likelihood for the Landau model on Day 15 data is {}.".format(landauLogLDay15))

❓ **Which model, Gaussian or Landau, fits the data better in terms of the log-likelihood?**

✳️ **Answer:** 

❓ **Given that the Gaussian model is a special case of the Landau distribution, why does your answer to the last question have to be the case, no matter what the data were?** *Hint: Imagine you find the parameters of a simple model that best fit your data, and then you make a more complicated model by adding an extra parameter.  You add the new parameter in such a way that one of its possible values produces the same fit as the simpler model.  What do you know about the new fit?*

✳️ **Answer:** 

This logic means that our better fit with the more complicated model is not so impressive.  In other language we have encountered in this class, it's possible that the new, more complicated model is simply *overfitting*: fitting noise in the data.

To determine if the extra parameter is "worth the trouble", we will compute the Bayesian information criterion (BIC).  Recall that the BIC includes a penalty on more complicated models, essentially setting a bar for *how much* better a model with extra parameters must fit the data to be statistically favored.

As we saw in lecture, the BIC is defined as
$$
\textrm{BIC} = L - \frac{1}{2} N_{params} \log(N_{datapoints}),
$$
where $L$ is the log-likelihood, $N_{params}$ is the number of free parameters in the model, and $N_{datapoints}$ is the number of datapoints to which the model was fit.

The following code computes BIC for the Gaussian and Landau models.

In [ ]:
# compute bic for gaussian model
numParamsGaussian = 2
numDatapoints = 16
gaussianBICDay15 = gaussianLogLDay15 - numParamsGaussian/2 * np.log(numDatapoints)

# compute bic for landau model
numParamsLandau = 3
numDatapoints = 16
landauBICDay15 = landauLogLDay15 - numParamsLandau/2 * np.log(numDatapoints)

Using this definition of BIC, having a BIC that is larger is indicative of evidence favoring that hypothesis;
a BIC difference of more than 3 is considered strong evidence in favor of that hypothesis;
and a BIC difference of more than 5 is considered very strong evidence.

(See for instance Table 6 of [this reference](https://www.jstor.org/stable/271063).  Note that some authors define BIC to be twice the value we consider here, and/or use the opposite sign.)

❓ **Which model is selected by BIC for bees of age 15 days?  Is one model strongly favored over the other?  How do you interpret this in terms of the number of distinct groups of bees?**

In [ ]:
# ✳️ **Answer:**

❓ **Repeat the analysis, including plots of the fit distributions and model selection using BIC, on data from bees of age 1 day.  Interpret your results.**

In [ ]:
# ✳️ **Answer:**